<a href="https://colab.research.google.com/github/Mc-Landers/Data-Science-Training-Python-Part-2/blob/main/LastNameFirstName_FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to AI: Machine Learning for Business - Final project

## Context

Congratulations, you’ve just been hired as an AI expert at a company.

For this final project you are required to choose and define a business problem of which you will apply machine learning to. 
Your final submission should include the following: 

- Problem definition
- Data Collection
- Data Processing
- Model Selection
- Results Visualization
- Results Interpretation



## What should you do ?

Make a notebook telling interesting things about the data you have fetched, tell a story (or many) using everything you learned. Build predictive models and compare them.  
You have to submit at least a notebook and any resources you used (like images or any other files).

## Assessment 
Here are the criteria we will use to assess your work:

### Is it meaningful?
As a machine learning expert you have to produce something meaningful enough, just plotting random data is not going to work. Like a story your analysis should have some kind of logical progression.

### How well did you use the technical knowledge you’ve been taught?
Obviously, the way you use everything you learned during the lectures is going to be assessed.

### Cleanliness, aesthetics and clearness of your notebook
Is your analysis full of unused code? Is it difficult to read? Have you tried to make it easy and enjoyable to read?

### Innovation
Creativity, surprising things or any good initiatives you take are potential bonus points.


### Careful:
	This work is individual, plagiarism is going to be measured by both machines and humans. Too many similarities between your work and any online or python buddy work will result in grade penalties.


Good Luck!


In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("AB_NYC_2019.csv")

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
df.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(38821, 16)

In [ ]:
df['neighbourhood_group'].nunique() # 5 groups for OneHotEncoder

5

In [ ]:
df.corr()['price']

id                               -0.006696
host_id                           0.006263
latitude                          0.031344
longitude                        -0.155298
price                             1.000000
minimum_nights                    0.025501
number_of_reviews                -0.035924
reviews_per_month                -0.030623
calculated_host_listings_count    0.052895
availability_365                  0.078276
Name: price, dtype: float64

select some of the useful columns. to not ruin the original dataset, i use a new one called df_test.

In [ ]:
df_test=df[['neighbourhood_group','latitude','longitude','room_type','minimum_nights','reviews_per_month','availability_365','price']]

here i use LabelEncoder for stratifying the dataset later.

In [ ]:
from sklearn.preprocessing import LabelEncoder
neighbor_cat=df_test['neighbourhood_group'].values
le=LabelEncoder()
neighbor_cat_encoded=le.fit_transform(neighbor_cat)
df_test.loc[:,'neighbor_cat']=neighbor_cat_encoded

room_cat=df_test['room_type'].values
le2=LabelEncoder()
room_cat_encoded=le2.fit_transform(room_cat)
df_test.loc[:,'room_cat']=room_cat_encoded

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [ ]:
df_test.head()

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,reviews_per_month,availability_365,price,neighbor_cat,room_cat
0,Brooklyn,40.64749,-73.97237,Private room,1,0.21,365,149,1,1
1,Manhattan,40.75362,-73.98377,Entire home/apt,1,0.38,355,225,2,0
3,Brooklyn,40.68514,-73.95976,Entire home/apt,1,4.64,194,89,1,0
4,Manhattan,40.79851,-73.94399,Entire home/apt,10,0.10,0,80,2,0
5,Manhattan,40.74767,-73.97500,Entire home/apt,3,0.59,129,200,2,0


stratify the dataset ( i think it's better than train-split-test)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_test, df_test[['neighbor_cat','room_cat']]):
    strat_train_set = df_test.iloc[train_index]
    strat_test_set = df_test.iloc[test_index]
strat_test_set.head()

,neighbourhood_group,latitude,longitude,room_type,minimum_nights,reviews_per_month,availability_365,price,neighbor_cat,room_cat
10220,Queens,40.74447,-73.92547,Private room,1,0.02,0,50,3,1
31699,Brooklyn,40.70666,-73.94379,Private room,4,0.29,0,60,1,1
17390,Brooklyn,40.68141,-73.96268,Entire home/apt,1,0.08,0,225,1,0
23200,Brooklyn,40.67731,-73.94616,Entire home/apt,5,0.35,78,150,1,0
30108,Brooklyn,40.67616,-73.96466,Entire home/apt,3,0.41,0,69,1,0


In [ ]:
strat_train_set=strat_train_set.drop(columns=['neighbor_cat','room_cat'])
strat_test_set=strat_test_set.drop(columns=['neighbor_cat','room_cat'])

get the train set and test set.

In [ ]:
X_train,y_train=strat_train_set.iloc[:,:-1].values,strat_train_set.iloc[:,-1].values
X_test,y_test=strat_test_set.iloc[:,:-1].values,strat_test_set.iloc[:,-1].values

feature scaling for numeric values.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ct = ColumnTransformer([
        ('somename', StandardScaler(), [1,2,4,5,6])
    ], remainder='passthrough')
X_train=ct.fit_transform(X_train)

encode the categorical values.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1,-2])], remainder='passthrough')
X_train=ct.fit_transform(X_train)

In [ ]:
X_train[0]

array([0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.560648208819058,
       -0.02298976295024965, -0.22408716148154975, -0.2203957627018006,
       -0.8334678596438506], dtype=object)

encode for test set data, here the indices for columns is a bit different because in the train set, i used feature scaling and after that the sequence of columns had been changed.

In [ ]:
ct2 = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3,0])], remainder='passthrough')
X_test=ct2.fit_transform(X_test)

now i got the data prepared and began to try some models.  
first of all, of course, liner model.

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [ ]:
lr.score(X_test,y_test)

-363.828817907866

don't panic. i knew from beginning that linear regression wouldn't work. but still, never expected that it would be this bad.  
so i tried random forest.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest=RandomForestRegressor()
forest.fit(X_train,y_train)

RandomForestRegressor()

In [ ]:
forest.score(X_test,y_test)

-4.963747107775436

still sucks, but much better compared to the linear model.  
so i used grid search to try to find a better parameter to see if it could be greater than 0.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': list(range(51,80)), 'max_features': ['auto','sqrt',8,10]},
    {'bootstrap': [False], 'n_estimators': [list(range(5,50))], 'max_features': ['auto','sqrt']},
  ]
forest_reg = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 590.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 392, in fit
    self._validate_estimator()
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_base.py", line 133, in _validate_estimator
    type(self.n_estimators)
ValueError: n_estimators must be an integer, got <class 

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid=[{'max_features': ['auto', 'sqrt', 8, 10],
                          'n_estimators': [51, 52, 53, 54, 55, 56, 57, 58, 59,
                                           60, 61, 62, 63, 64, 65, 66, 67, 68,
                                           69, 70, 71, 72, 73, 74, 75, 76, 77,
                                           78, 79]},
                         {'bootstrap': [False],
                          'max_features': ['auto', 'sqrt'],
                          'n_estimators': [[5, 6, 7, 8, 9, 10, 11, 12, 13, 14,
                                            15, 16, 17, 18, 19, 20, 21, 22, 23,
                                            24, 25, 26, 27, 28, 29, 30, 31, 32,
                                            33, 34, ...]]}],
             return_train_score=True, scoring='neg_mean_squared_error')

i don't run this one because it takes a while even on my own computer, i think it would be even slower in google colab. i tried several combinations of paramters in grid search, and this block of code gives me the best result, which is, unfortunately, -0.78.

here i tried XGBoost, without fine tuning the parameters, its score is -0.61. i believe that we can make it greater than 0 but usually a good score should be greater than 0.8 or even 0.9.

In [ ]:
import xgboost as xgb
bst=xgb.XGBRegressor(objective ='reg:squarederror')
bst.fit(X_train,y_train)

XGBRegressor(objective='reg:squarederror')

In [ ]:
bst.score(X_test,y_test)

-0.6198665484279233

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'max_depth': list(range(3,10,2)), 'mean_child_weight': list(range(1,6,2))},
  ]
bst = xgb.XGBRegressor(base_score=0.5, colsample_bylevel=1,
             gamma=0, learning_rate=0.1,
             max_delta_step=0, missing=None,
             n_estimators=100, nthread=-1,
             reg_alpha=0,reg_lambda=1, scale_pos_weight=1,silent=True, 
             subsample=1, objective ='reg:squarederror',)

grid_search2 = GridSearchCV(bst, param_grid, cv=5,
                           scoring='r2', return_train_score=True,n_jobs=-1)
grid_search2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(nthread=-1, objective='reg:squarederror',
                                    silent=True),
             n_jobs=-1,
             param_grid=[{'max_depth': [3, 5, 7, 9],
                          'mean_child_weight': [1, 3, 5]}],
             return_train_score=True, scoring='r2')

In [ ]:

grid_search2.best_estimator_.score(X_test,y_test)

-0.6198665484279233

# ⭐ New Dataset



find the new one in google drive and upload it on google colab!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

Here I cleaned the data.

In [ ]:
import pandas as pd
df=pd.read_csv('car_prices.csv',error_bad_lines=False,warn_bad_lines=True)

df=df.dropna() # i checked the na values, most of them are not important so I simply dropped them

# compute the age of the car when it is sold. makes more sense to me compared to the 'year' attribute
df['sale_date'] = pd.to_datetime(df['saledate'],utc=True)
df['sale_year']=df['sale_date'].dt.year
df['age']=df['sale_year']-df['year']

df['make']=df['make'].map(lambda x:str(x).lower()) # all characters would be lower-case so that there's less 'nuinque'
df['model']=df['model'].map(lambda x:str(x).lower())
df['trim']=df['trim'].map(lambda x:str(x).lower())
df['body']=df['body'].map(lambda x:str(x).lower())

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 408163: expected 16 fields, saw 17\nSkipping line 417837: expected 16 fields, saw 17\nSkipping line 421291: expected 16 fields, saw 17\nSkipping line 424163: expected 16 fields, saw 17\n'
b'Skipping line 427042: expected 16 fields, saw 17\nSkipping line 427045: expected 16 fields, saw 17\nSkipping line 434426: expected 16 fields, saw 17\nSkipping line 444503: expected 16 fields, saw 17\nSkipping line 453796: expected 16 fields, saw 17\n'
b'Skipping line 461599: expected 16 fields, saw 17\nSkipping line 461

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,sale_date,sale_year,age
0,2015,kia,sorento,lx,suv,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,"kia motors america, inc",20500,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),2014-12-16 04:30:00+00:00,2014,-1
1,2015,kia,sorento,lx,suv,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,"kia motors america, inc",20800,21500,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),2014-12-16 04:30:00+00:00,2014,-1
2,2014,bmw,3 series,328i sulev,sedan,automatic,wba3c1c51ek116351,ca,4.5,1331.0,gray,black,financial services remarketing (lease),31900,30000,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),2015-01-14 20:30:00+00:00,2015,1
3,2015,volvo,s60,t5,sedan,automatic,yv1612tb4f1310987,ca,4.1,14282.0,white,black,volvo na rep/world omni,27500,27750,Thu Jan 29 2015 04:30:00 GMT-0800 (PST),2015-01-28 20:30:00+00:00,2015,0
4,2014,bmw,6 series gran coupe,650i,sedan,automatic,wba6b2c57ed129731,ca,4.3,2641.0,gray,black,financial services remarketing (lease),66000,67000,Thu Dec 18 2014 12:30:00 GMT-0800 (PST),2014-12-18 04:30:00+00:00,2014,0


### First, we dropped all the rows with 'NA' values. Then, we extracted a new variable from Sale Date and Year. This is because having an "Age" attribute may prove to be more important for the predictions than having a "Year" attribute. For uniformity, we changed all characters in the dataset to lowercase.
### The dataset includes the following variables:


*   Year (Model year)
*   Make (Car brand)
*   Model (Car edition)
*   Trim (Car trim level; the model version)
*   Body (Car shape/body type)
*   Transmission (Power mechanism in the car)
*   VIN (Vehicle Identification Number)
*   State (The state where the car was being auctioned)
*   Condition (Car condition at time of auction)
*   Odometer (Car mileage)
*   Color (Exterior colour)
*   Interior (Car interior colour)
*   Seller (Car seller)
*   MMR (Manheim Market Report)
*   Selling price (The price at which the car was sold at the auction)
*   Sale Date (The date the price was auctioned)
*   Sale Year (The year the car was auctioned; extracted from the Sale Date variable)
*   Age (The age of the car at time of auction; extracted by subtracting the Year from the Sale Year)



In [ ]:
df['mmr'].value_counts

<bound method IndexOpsMixin.value_counts of 0         20500
1         20800
2         31900
3         27500
4         66000
          ...  
558805    20300
558807    30200
558808    29800
558809    15100
558810    29600
Name: mmr, Length: 472336, dtype: int64>

In [ ]:
# this is my version of feature selection, we could always have more options. i dropped 'mmr' because it's already a predicted price made by others.
df=df.drop(columns=['vin','seller','mmr','sale_date','sale_year','saledate','year'])

# some of 'age' are -1 for reasons i dont know, but not a big number, so i just dropped them
df=df.loc[df['age']>=0,:]

df.head()

,make,model,trim,body,transmission,state,condition,odometer,color,interior,sellingprice,age
2,bmw,3 series,328i sulev,sedan,automatic,ca,4.5,1331.0,gray,black,30000.0,1
3,volvo,s60,t5,sedan,automatic,ca,4.1,14282.0,white,black,27750.0,0
4,bmw,6 series gran coupe,650i,sedan,automatic,ca,4.3,2641.0,gray,black,67000.0,0
6,bmw,m5,base,sedan,automatic,ca,3.4,14943.0,black,black,65000.0,0
7,chevrolet,cruze,1lt,sedan,automatic,ca,2.0,28617.0,black,black,9800.0,0


### We deleted some features. VIN and Seller were deleted because their values are too unique to hold any relevance for the dataset. Sale Date, Sale Year, and Year were deleted because the new "Age" variable tells a more concise story. Since MMR is already a price prediction in its own right, we got rid of it as well.
### Some of the car ages were negative numbers, so we dropped them.

In [ ]:
df2=df.copy()

**We made a copy so that the original one won't be ruined.**

### data shuffling

In [ ]:
import numpy as np
# reindex the order of the columns to make it more convinient later
column_titles=['make', 'model', 'trim', 'body', 'transmission', 'state','color', 'interior',  'age','condition','odometer','sellingprice']
df2 = df2.reindex(columns=column_titles)

# prepare for shuffling
df2['age_cat'] = np.ceil(df2['age'] / 1.5)
df2['age_cat'].where(df2['age_cat'] < 10, 10.0, inplace=True)
df2['condition_cat']=round(df2['condition'],0)
df2['odometer_cat']=np.ceil(df2['odometer']/100000)
df2['odometer_cat'].where(df2['odometer_cat'] < 2, 2.0, inplace=True)

df2.head()

NameError: ignored

In [ ]:
df2['trim'].nunique()

1123

In [ ]:
df2['model'].nunique()

662

In [ ]:
df2['state'].nunique()

NameError: ignored

here comes the classic one (in my thought). i didn't do this first, and it took me more than an hour to realize there's something wrong with our individual homework. we had to use onehotencoder first for this dataset.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse=False),['make', 'model', 'trim', 'body', 'transmission', 'state','color', 'interior'])], remainder='passthrough')
nparray=ct.fit_transform(df2)

# i transformed the result to dataframe cuz i don't know if numpy array could be directly shuffled.
df3=pd.DataFrame(nparray)
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.5,1331.0,30000.0,1.0,4.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,14282.0,27750.0,0.0,4.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3,2641.0,67000.0,0.0,4.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4,14943.0,65000.0,0.0,3.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,28617.0,9800.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.9,39821.0,11900.0,2.0,4.0,1.0
63411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.6,18729.0,13500.0,2.0,3.0,1.0
63412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.5,36836.0,16400.0,2.0,4.0,1.0
63413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.7,30233.0,15100.0,2.0,4.0,1.0


stratified shuffling

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df3, df3.iloc[:,-3:]):
    strat_train_set = df3.iloc[train_index]
    strat_test_set = df3.iloc[test_index]

strat_test_set

NameError: ignored

In [ ]:
# drop the 'cat' columns we used for shuffling
strat_train_set=strat_train_set.drop(columns=[2427,2428,2429])
strat_test_set=strat_test_set.drop(columns=[2427,2428,2429])

just to check

In [ ]:
strat_test_set

now we have our training set and test set

In [ ]:
X_train,y_train=strat_train_set.iloc[:,:-1].values,strat_train_set.iloc[:,-1].values
X_test,y_test=strat_test_set.iloc[:,:-1].values,strat_test_set.iloc[:,-1].values

### Feature Scaling

In [ ]:
from scipy._lib.six import X
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
scaler=StandardScaler()
num_attribs=[2423,2424,2425]
ct=ColumnTransformer([
        ("num", StandardScaler(), num_attribs),
    ],remainder='passthrough')
X_train=ct.fit_transform(X_train)
X_train

here i did feature scaling on our test set, using the same scaler with the training set so that the test set would stay pure. **imortant: use .transform rather than .fit_transform!!**

In [ ]:
X_test=ct.transform(X_test)

### model testing

Aight, now we have our data prepared and could run some models

#### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
# lol this one is a DISASTER
lr.score(X_test,y_test)

#### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest=RandomForestRegressor(n_jobs=-1)
# this one takes forever
forest.fit(X_train,y_train)

In [ ]:
forest.score(X_train,y_train)

In [ ]:
forest.score(X_test,y_test)

#### XGBoost

In [ ]:
import xgboost as xgb
bst=xgb.XGBRegressor(objective ='reg:squarederror',n_jobs=-1)
bst.fit(X_train,y_train)

In [ ]:
bst.score(X_train,y_train)

In [ ]:
bst.score(X_test,y_test)